In [2]:
import pandas as pd
import numpy as np


In [5]:
ARRAY_COLUMNS = ['description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca']


In [12]:
sample_train = pd.read_csv('../data_20220831/sample_train_processed_processed.csv').drop('Unnamed: 0', axis=1)
sample_train

,cv_id,jd_id,model_label,age,degree_index_cv,degree_index_jd,degree_match_cvjd,description_tfidf_pca,equal_job,gender_index_cv,...,min_annual_salary,parsed_desiredSalary,position_tfidf_pca,recruit_number,requirement_tfidf_pca,school_intersect_cvjd,title_category_tags_tfidf_pca,workYear,work_duration_mean,workyear_match_cvjd
0,2c9207158059e82b018067f52ee520f1,2c9207157ef60164017ef604e195022e,0,26.0,5,5,1,"[0.0034318132226696, 0.0453854119444269, -0.06...",3,2,...,400.0,180.0,"[0.8020301121767343, 0.4229101266796492, -0.06...",1,"[0.0146099317606446, -0.0973321531897229, -0.0...",1,"[-0.0268220597366096, 0.0146753642354594, -0.1...",4.0,270.00,1
1,2c920715804c9ef601805374e101570a,2c92071580b3a1010180bda0e1166bfd,0,46.0,5,5,1,"[0.0526010120409367, 0.1687600744043454, 0.042...",5,2,...,960.0,360.0,"[0.1703266952313507, -0.0428909764484499, 0.01...",1,"[-0.0804330166663323, -0.265194138646796, 0.05...",1,"[-0.1403481086160699, 0.101545233205827, -0.14...",23.0,1019.50,1
2,2c9207157ef7ed7f017f075fc6640e79,2c9207157d2f0ec5017d5a854c143177,0,36.0,5,5,1,"[-0.0798424978631493, 0.0294367690081744, -0.0...",3,0,...,350.0,360.0,"[-0.0064237143650397, -0.1771169508284544, 0.0...",1,"[-0.0967045533557524, -0.0158876913753405, -0....",0,"[0.0041959442715183, -0.0689690014332659, -0.0...",12.0,832.00,0
3,8a69c4e9824fd55f018258b725082f07,8a69f68e82198e9101822545359e0a0d,0,28.0,5,-3,1,"[0.0953569788219484, -0.1217749783967515, 0.06...",4,0,...,480.0,360.0,"[-0.1512166410895448, 0.039388905830567, -0.01...",1,"[0.1213571418999401, 0.0752965052240779, -0.04...",1,"[0.0593428963865967, -0.0521583781812485, -0.0...",5.0,401.40,1
4,2c920715800e8898018022a768d41cc3,2c9207157f347800017f72dd6e04248b,0,33.0,5,5,1,"[0.0072163480609644, 0.0044387415783859, 0.036...",2,0,...,320.0,360.0,"[0.8020301121767343, 0.4229101266796492, -0.06...",1,"[-0.0555047818278877, -0.0795936243595016, 0.0...",1,"[-0.4085584894502868, 0.2043626298020381, 0.02...",8.0,692.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58499,2c9207157e58b769017e703d68483561,2c9207157dc878e5017ddab2b223521b,1,32.0,2,-3,1,"[-0.0849161155867982, -0.0575842157103247, -0....",0,0,...,96.0,360.0,"[-0.0368041780176452, -0.0857596688737132, -0....",15,"[-0.0668460252602592, 0.0700903434401426, 0.05...",1,"[0.0196958641886768, -0.0692635968080485, -0.0...",8.0,783.50,0
58500,8a69c4e88262c85901826bc2582c33d0,8a69f61482627e7a0182628269db2b56,1,32.0,5,5,1,"[0.0670356722750115, -0.1574484192011676, 0.00...",4,0,...,154.0,270.0,"[-0.1453312134516913, 0.0613943686252203, -0.0...",6,"[-0.0098772535847565, 0.1301945044403502, -0.0...",1,"[0.1167680749390213, -0.0366297624844817, -0.0...",9.0,814.50,1
58501,2c9207157ef7ed7f017f0a990d4d23a0,2c9207157ef7ed16017f0c28fa6c5e48,1,42.0,5,5,1,"[-0.1671116181744903, 0.0182776948093178, 0.15...",4,0,...,480.0,360.0,"[0.0114686317604377, -0.2795560531646292, 0.07...",1,"[-0.097456235834622, -0.0709741819063322, 0.27...",1,"[-0.0871293740917814, -0.0944614800646356, 0.1...",13.0,1141.75,1
58502,8a69d7a68219933d01821fc0c69843ea,2c9207158114fb2901815c03bb257372,1,30.0,5,5,1,"[0.1475688107160124, -0.1547690605224491, 0.02...",4,0,...,280.0,600.0,"[-0.0878304289432486, -0.0431864975598747, -0....",1,"[0.1227189818078986, 0.0444096728787962, -0.01...",1,"[0.1580939681081193, 0.0353872320416606, -0.09...",6.0,501.50,1


In [13]:
def school_lables_to_list(series, school_col):
    school_labels = series[school_col]
    school_labels_list = [float(item) for item in school_labels.strip('[]').replace(' ', '').split(',')]
    if school_labels_list[0] == '':
        return []
    return school_labels_list
sample_train['description_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('description_tfidf_pca', ))
sample_train['position_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('position_tfidf_pca', ))
sample_train['requirement_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('requirement_tfidf_pca', ))
sample_train['kills_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('kills_tfidf_pca', ))
sample_train['title_category_tags_tfidf_pca'] = sample_train.apply(school_lables_to_list, axis=1, args=('title_category_tags_tfidf_pca', ))

In [19]:
sample_train.to_pickle('../data_20220831/sample_train_processed_processed.pkl')

In [20]:
# 以后都读取pickle的数据格式
sample_train_pickle = pd.read_pickle('../data_20220831/sample_train_processed_processed.pkl')

In [31]:
sample_train_pickle.columns

Index(['cv_id', 'jd_id', 'model_label', 'age', 'degree_index_cv',
       'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca',
       'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU',
       'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary',
       'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca',
       'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd',
       'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean',
       'workyear_match_cvjd'],
      dtype='object')

In [57]:
sample_train_pickle['workyear_match_cvjd'][0].__class__

numpy.int64